In [1]:
import xarray as xr
from netCDF4 import Dataset, num2date
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import xesmf as xe

In [20]:
tp_era5 = xr.open_dataset('Final_Data/ERA5_Total_Precipitation_2023120106_2024081606.nc').tp
tp_era5_USA = xr.open_dataset('Final_Data/ERA5_Total_Precipitation_USA_2023120106_2024081606.nc').tp
tp_s2s = xr.open_dataset('S2S_Data/precip_perturbed.nc').tp

In [3]:
tp_era5

<xarray.DataArray 'tp' (forecast_initial_time: 1036, latitude: 721,
                        longitude: 1440)> Size: 4GB
[1075616640 values with dtype=float32]
Coordinates:
  * forecast_initial_time  (forecast_initial_time) datetime64[ns] 8kB 2023-12...
  * latitude               (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude              (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
Attributes:
    long_name:           total precipitation
    short_name:          tp
    units:               m
    original_format:     WMO GRIB 1 with ECMWF local table
    ecmwf_local_table:   NaN
    ecmwf_parameter:     NaN
    minimum_value:       <xarray.DataArray ()> Size: 4B\narray(0., dtype=floa...
    maximum_value:       <xarray.DataArray ()> Size: 4B\narray(0.06579924, dt...
    grid_specification:  0.25 degree x 0.25 degree from 90N to 90S and 0E to ...

In [4]:
regridder = xe.Regridder(tp_era5, tp_s2s, 'bilinear')

In [5]:
tp_era5_regrid = regridder(tp_era5)

In [6]:
tp_era5_regrid

<xarray.DataArray (forecast_initial_time: 1036, latitude: 121, longitude: 240)> Size: 120MB
array([[[1.57356262e-05, 1.57356262e-05, 1.57356262e-05, ...,
         1.57356262e-05, 1.57356262e-05, 1.57356262e-05],
        [7.15255737e-06, 6.19888306e-06, 6.19888306e-06, ...,
         6.67572021e-06, 6.67572021e-06, 6.67572021e-06],
        [9.53674316e-06, 1.04904175e-05, 1.04904175e-05, ...,
         4.29153442e-06, 5.72204590e-06, 8.58306885e-06],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.35898590e-05, 1.35898590e-05, 1.35898590e-05, ...,
         1.35898590e-05, 1.35898590e-05, 1.35898590e-05],
        [6.67572021e-06, 6.67572021e-06, 6.91413879e-06, ...,
         3.33786011e-06, 5.24520874e-06, 5.24520874e-06],
        [1.76429749e-05, 1.62124634e-05, 1.66893005e-05, ...,
         2.09808350e-05, 1.90734863e-05, 1.97887421e-05],
...
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[2.78472900e-04, 2.78472900e-04, 2.78472900e-04, ...,
         2.78472900e-04, 2.78472900e-04, 2.78472900e-04],
        [4.64916229e-05, 4.98294830e-05, 5.24520874e-05, ...,
         3.67164612e-05, 3.93390656e-05, 4.33921814e-05],
        [1.57356262e-05, 1.52587891e-05, 1.43051147e-05, ...,
         3.43322754e-05, 2.83718109e-05, 2.16960907e-05],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]], dtype=float32)
Coordinates:
  * forecast_initial_time  (forecast_initial_time) datetime64[ns] 8kB 2023-12...
    surface                float64 8B 0.0
  * latitude               (latitude) float64 968B 90.0 88.5 ... -88.5 -90.0
  * longitude              (longitude) float64 2kB 0.0 1.5 3.0 ... 357.0 358.5
Attributes:
    regrid_method:  bilinear

In [26]:
lat_min = tp_era5_USA.latitude.min().item()
lat_max = tp_era5_USA.latitude.max().item()
lon_min = tp_era5_USA.longitude.min().item()
lon_max = tp_era5_USA.longitude.max().item()

# Select the subset of tp_s2s that overlaps with tp_era5_USA
tp_s2s_USA = tp_s2s.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))

In [30]:
print(lat_max, lat_min, lon_min, lon_max)

60.0 15.0 225.0 305.0


In [27]:
tp_era5_USA

<xarray.DataArray 'tp' (forecast_initial_time: 1036, latitude: 181,
                        longitude: 321)> Size: 241MB
[60192636 values with dtype=float32]
Coordinates:
  * forecast_initial_time  (forecast_initial_time) datetime64[ns] 8kB 2023-12...
  * latitude               (latitude) float64 1kB 60.0 59.75 59.5 ... 15.25 15.0
  * longitude              (longitude) float64 3kB 225.0 225.2 ... 304.8 305.0
Attributes:
    long_name:           total precipitation
    short_name:          tp
    units:               m
    original_format:     WMO GRIB 1 with ECMWF local table
    ecmwf_local_table:   NaN
    ecmwf_parameter:     NaN
    minimum_value:       <xarray.DataArray ()> Size: 4B\narray(0., dtype=floa...
    maximum_value:       <xarray.DataArray ()> Size: 4B\narray(0.06579924, dt...
    grid_specification:  0.25 degree x 0.25 degree from 90N to 90S and 0E to ...

In [28]:
tp_s2s_USA

<xarray.DataArray 'tp' (number: 20, time: 6, step: 185, latitude: 31,
                        longitude: 54)> Size: 149MB
[37162800 values with dtype=float32]
Coordinates:
  * number      (number) int64 160B 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * time        (time) datetime64[ns] 48B 2024-01-01 2024-02-01 ... 2024-06-01
  * step        (step) timedelta64[ns] 1kB 0 days 00:00:00 ... 46 days 00:00:00
    surface     float64 8B ...
  * latitude    (latitude) float64 248B 60.0 58.5 57.0 55.5 ... 18.0 16.5 15.0
  * longitude   (longitude) float64 432B 225.0 226.5 228.0 ... 301.5 303.0 304.5
    valid_time  (time, step) datetime64[ns] 9kB ...
Attributes: (12/31)
    GRIB_paramId:                             228228
    GRIB_dataType:                            pf
    GRIB_numberOfPoints:                      29040
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            accum
    ...                                       ...
    GRIB_shortName:                           tp
    GRIB_totalNumber:                         101
    GRIB_units:                               kg m**-2
    long_name:                                Total Precipitation
    units:                                    kg m**-2
    standard_name:                            unknown